In [1]:
import pandas as pd
from glob import glob
from tqdm.notebook import tqdm

BASE = "/media/giani/Gianicosas/Magister/Proyecto/raw_data"

# Proceso de matricula

In [7]:
# Fuente: https://www.mifuturo.cl/bases-de-datos-de-matriculados/
proceso = pd.read_csv(f"{BASE}/OFICIAL_WEB_PROCESO_MAT_2007_AL_2022_24_06_2022_E.csv", encoding="latin-1", sep=";")
proceso.head()

/tmp/ipykernel_25481/389441206.py:1: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{BASE}/OFICIAL_WEB_PROCESO_MAT_2007_AL_2022_24_06_2022_E.csv", encoding="latin-1", sep=";")


,AÑO,TOTAL MATRICULADOS,MATRICULADOS MUJERES POR PROGRAMA,MATRICULADOS HOMBRES POR PROGRAMA,TOTAL MATRICULADOS PRIMER AÑO,MATRICULADOS MUJERES PRIMER AÑO,MATRICULADOS HOMBRES PRIMER AÑO,CLASIFICACIÓN INSTITUCIÓN NIVEL 1,CLASIFICACIÓN INSTITUCIÓN NIVEL 2,CLASIFICACIÓN INSTITUCIÓN NIVEL 3,...,TES PARTICULAR SUBVENCIONADO,TES PARTICULAR PAGADO,TES CORP. DE ADMINISTRACIÓN DELEGADA,TES SERVICIO LOCAL EDUCACION,TOTAL TES,% COBERTURA TES,TIPO ESTABLECIMIENTO HC,TIPO ESTABLECIMIENTO TP,CLAS_EST ADULTO,CLAS_EST JOVEN
0,MAT_2022,2,2.0,NaN,NaN,NaN,NaN,Institutos Profesionales,Institutos Profesionales,Institutos Profesionales,...,2.0,NaN,NaN,NaN,2,"100,0%",2.0,NaN,NaN,2.0
1,MAT_2022,10,8.0,2.0,NaN,NaN,NaN,Institutos Profesionales,Institutos Profesionales,Institutos Profesionales,...,3.0,2.0,NaN,NaN,9,"90,0%",4.0,5.0,2.0,7.0
2,MAT_2022,34,NaN,34.0,NaN,NaN,NaN,Institutos Profesionales,Institutos Profesionales,Institutos Profesionales,...,14.0,3.0,NaN,NaN,26,"76,5%",17.0,9.0,4.0,22.0
3,MAT_2022,10,NaN,10.0,NaN,NaN,NaN,Institutos Profesionales,Institutos Profesionales,Institutos Profesionales,...,4.0,NaN,NaN,NaN,10,"100,0%",6.0,4.0,1.0,9.0
4,MAT_2022,76,4.0,72.0,NaN,NaN,NaN,Institutos Profesionales,Institutos Profesionales,Institutos Profesionales,...,29.0,6.0,4.0,NaN,71,"93,4%",30.0,41.0,4.0,67.0


# Matriculados

### Todas las columnas
```python
# pd.read_csv(f"{BASE}/20220719_Matrícula_Ed_Superior_2022_PUBL_MRUN.csv", nrows=5, sep=";").columns
[
    'cat_periodo', 'id', 'codigo_unico', 'mrun', 'gen_alu', 'fec_nac_alu', 'rango_edad', 'anio_ing_carr_ori',
    'sem_ing_carr_ori', 'anio_ing_carr_act', 'sem_ing_carr_act', 'anio_ing_carr_act', 'tipo_inst_1', 'tipo_inst_2', 'tipo_inst_3',
    'cod_inst', 'nomb_inst', 'cod_sede', 'nomb_sede', 'cod_carrera', 'nomb_carrera', 'modalidad', 'jornada',
    'version', 'tipo_plan_carr', 'dur_estudio_carr', 'dur_proceso_tit', 'dur_total_carr', 'region_sede',
    'provincia_sede', 'comuna_sede', 'nivel_global', 'nivel_carrera_1', 'nivel_carrera_2', 'requisito_ingreso',
    'vigencia_carrera', 'formato_valores', 'valor_matricula', 'valor_arancel', 'codigo_demre', 'area_conocimiento',
    'cine_f_97_area', 'cine_f_97_subarea', 'area_carrera_generica', 'cine_f_13_area', 'cine_f_13_subarea',
    'acreditada_carr', 'acreditada_inst', 'acre_inst_desde_hasta', 'acre_inst_anio', 'costo_proceso_titulacion',
    'costo_obtencion_titulo_diploma', 'forma_ingreso'
]
```

### Cols para determinar vector objetivo
```python
['cat_periodo', 'mrun', 'anio_ing_carr_ori', 'anio_ing_carr_act', 'tipo_inst_1', 'dur_estudio_carr', 'cod_carrera']
```

### Tipos de instituciones
```python
# Salida: ['Universidades', 'Institutos Profesionales', 'Centros de Formación Técnica']
pd.concat([
    pd.read_csv(f, sep=";", usecols=["tipo_inst_1"])
    for f in tqdm(glob(f"{BASE}/20220719_Matrícula_Ed_Superior_*.csv"), total=16)
])["tipo_inst_1"].unique()
instituciones = ['Institutos Profesionales', 'Centros de Formación Técnica']
```

## Lectura de datos matriculados

Fuente: https://datosabiertos.mineduc.cl/matricula-en-educacion-superior/

In [9]:
def iter_df_read(file, col_name, condition_list, chunksize=1000, **kwargs):
    iter_csv = pd.read_csv(file, iterator=True, chunksize=chunksize, **kwargs)
    return pd.concat([chunk[chunk[col_name].isin(condition_list)] for chunk in iter_csv])

In [11]:
cols_time = ['cat_periodo', 'mrun', 'anio_ing_carr_ori', 'anio_ing_carr_act', 'tipo_inst_1', 'dur_estudio_carr', 'cod_carrera']
instituciones = ['Institutos Profesionales', 'Centros de Formación Técnica']

matriculados = pd.concat([
    iter_df_read(f, "tipo_inst_1", instituciones, sep=";", usecols=cols_time)
    for f in tqdm(glob(f"{BASE}/20220719_Matrícula_Ed_Superior_*.csv"), total=16)
])
matriculados.head()

  0%|          | 0/16 [00:00<?, ?it/s]

,cat_periodo,mrun,anio_ing_carr_ori,anio_ing_carr_act,tipo_inst_1,dur_estudio_carr
1,2007,55.0,2006.0,2006.0,Institutos Profesionales,8.0
3,2007,132.0,2006.0,2006.0,Institutos Profesionales,4.0
4,2007,133.0,2007.0,2007.0,Institutos Profesionales,4.0
7,2007,213.0,2006.0,2006.0,Institutos Profesionales,8.0
12,2007,341.0,2006.0,2006.0,Centros de Formación Técnica,5.0


In [14]:
matriculados.isna().sum()

cat_periodo                0
mrun                    2109
anio_ing_carr_ori       7402
anio_ing_carr_act    3139917
tipo_inst_1                0
dur_estudio_carr         160
dtype: int64

In [16]:
matriculados["mrun"].value_counts().head()

7085589.0     39
10395662.0    22
21186499.0    22
14981897.0    20
16639641.0    20
Name: mrun, dtype: int64

In [17]:
matriculados[matriculados["mrun"]==7085589.0].sort_values(by=["mrun", "cat_periodo"]).head(10)

,cat_periodo,mrun,anio_ing_carr_ori,anio_ing_carr_act,tipo_inst_1,dur_estudio_carr
219226,2007,7085589.0,2007.0,2007.0,Centros de Formación Técnica,5.0
231025,2008,7085589.0,2008.0,2008.0,Centros de Formación Técnica,4.0
231026,2008,7085589.0,2008.0,2008.0,Institutos Profesionales,4.0
251901,2009,7085589.0,2009.0,NaN,Institutos Profesionales,8.0
251902,2009,7085589.0,2008.0,2008.0,Centros de Formación Técnica,4.0
251903,2009,7085589.0,2009.0,NaN,Centros de Formación Técnica,4.0
251904,2009,7085589.0,2009.0,2009.0,Centros de Formación Técnica,4.0
277492,2010,7085589.0,2010.0,NaN,Centros de Formación Técnica,4.0
277493,2010,7085589.0,2010.0,NaN,Institutos Profesionales,4.0
277494,2010,7085589.0,2010.0,2010.0,Institutos Profesionales,8.0


### Alumno desertor
- Tiene que haber pasado un tiempo igual o mayor a la duración de la carrera
- El último semestre cursado tiene que ser menor a la cantidad de semestres de la carrera
- Considerar todo para el mismo codigo de carrera cada vez